In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os
import glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import json
import cftime
from itertools import product
from cftime import DatetimeNoLeap
import esmvalcore.preprocessor
import xesmf as xe
from xmip.preprocessing import promote_empty_dims, broadcast_lonlat, replace_x_y_nominal_lat_lon, rename_cmip6
from tqdm import tqdm
import dask


In [3]:
### make a table of models showing what temporal resolution is available for each variable :

vars = ['tas', 'ta', 'hfss', 'vas', 'uas', 'sfcWind', 'rlds', 'rlus', 'ts', 'clivi', 'clwvi']

ordered_table_list = ['3hr', '6hr', '6hrPlev', '6hrPlevPt', 'day', 'CFday', 'Amon'] #highest to lowest preference
reversed_table_list = ordered_table_list.copy()
reversed_table_list.reverse()
### get full list of models (defined as models with 3hr/tas):

dirs=[]
for x in glob.glob('/badc/cmip6/data/CMIP6/CMIP/*/*/historical/r1i*/3hr/tas/*/latest/'): 
    dirs.append(x)

models = []
for x in dirs:
    model = x.split('/')[7]
    if model not in models:
        models.append(model)

#make a df
df = pd.DataFrame({'Model':models, 
                   })

for var in vars:
    
    ### make dict {table_id : list of models with var available in that table_id}
    models_list = []
    for table in ordered_table_list:   
            
        dirs=[]
        for x in glob.glob('/badc/cmip6/data/CMIP6/CMIP/*/*/historical/r1i*/{t}/{v}/*/latest/'.format(t=table, v=var)): 
            dirs.append(x)
        
        models = []
        for x in dirs:
            model = x.split('/')[7]
            if model not in models:
                models.append(model)
        models_list.append(models)
    
    tab_dict = dict(zip(ordered_table_list, models_list))

    ### use dict to fill up dataframe
    df[var] = np.nan 
    for table in reversed_table_list:        
        df[var] = df[var].mask(df['Model'].isin(tab_dict[table]), table)

df.to_csv('Highest_resolution_on_CEDA_for_vars.csv')
df


,Model,tas,ta,hfss,vas,uas,sfcWind,rlds,rlus,ts,clivi,clwvi
0,TaiESM1,3hr,6hrPlevPt,3hr,NaN,NaN,day,3hr,3hr,6hrPlevPt,Amon,Amon
1,AWI-CM-1-1-MR,3hr,Amon,3hr,3hr,3hr,Amon,3hr,3hr,Amon,NaN,Amon
2,AWI-ESM-1-1-LR,3hr,6hrPlevPt,3hr,3hr,3hr,day,3hr,3hr,6hrPlevPt,CFday,CFday
3,BCC-CSM2-MR,3hr,6hrPlevPt,3hr,3hr,3hr,Amon,3hr,3hr,Amon,Amon,Amon
4,FGOALS-g3,3hr,day,3hr,NaN,NaN,day,3hr,3hr,Amon,Amon,Amon
5,IITM-ESM,3hr,day,3hr,3hr,3hr,day,3hr,3hr,Amon,Amon,Amon
6,CanESM5,3hr,day,day,3hr,3hr,day,3hr,3hr,Amon,CFday,CFday
7,CMCC-CM2-SR5,3hr,6hrPlevPt,3hr,3hr,3hr,day,3hr,3hr,Amon,Amon,Amon
8,CMCC-ESM2,3hr,6hrPlevPt,3hr,3hr,3hr,6hrPlev,3hr,3hr,Amon,Amon,Amon
9,CNRM-CM6-1,3hr,day,3hr,3hr,3hr,day,3hr,3hr,Amon,CFday,CFday


In [20]:
### now show a table with a list of models with 3 hourly data for a minimum set of vars
min_vars = ['tas', 'ta', 'hfss', 'vas', 'uas', 'rlds', 'rlus']

df_min = df.drop(columns=[x for x in vars if x not in min_vars])
for var in min_vars:
    df_min = df_min[df_min[var].isin(['3hr', '6hrPlevPt'])]
print(len(df_min.Model.unique()))
mods_6hrly = df_min.Model.unique()
mods_6hrly

21


array(['AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'CMCC-CM2-SR5', 'CMCC-ESM2',
       'ACCESS-ESM1-5', 'ACCESS-CM2', 'EC-Earth3', 'EC-Earth3-AerChem',
       'EC-Earth3-Veg-LR', 'MPI-ESM-1-2-HAM', 'KIOST-ESM', 'MIROC6',
       'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'MPI-ESM1-2-HR',
       'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'GISS-E2-1-G', 'KACE-1-0-G',
       'GFDL-CM4', 'NESM3'], dtype=object)

In [6]:
### now show a table with a list of models with 3 hourly data for an extreme minimum set of vars

extra_min_vars = ['tas', 'ta', 'rlds', 'rlus']

df_min = df.drop(columns=[x for x in vars if x not in extra_min_vars])
for var in extra_min_vars:
    df_min = df_min[df_min[var].isin(['3hr', '6hrPlevPt'])]
print(len(df_min.Model.unique()))
df_min.Model.unique()

21


array(['AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'CMCC-CM2-SR5', 'CMCC-ESM2',
       'ACCESS-ESM1-5', 'ACCESS-CM2', 'EC-Earth3', 'EC-Earth3-AerChem',
       'EC-Earth3-Veg-LR', 'MPI-ESM-1-2-HAM', 'KIOST-ESM', 'MIROC6',
       'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'MPI-ESM1-2-HR',
       'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'GISS-E2-1-G', 'KACE-1-0-G',
       'GFDL-CM4', 'NESM3'], dtype=object)